In [11]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import os
import pandas as pd
import re
import sqlalchemy as sa
import sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('main.ipynb'))))

import database as db
from models import DataTrustedIdentifier

## Database connection

In [13]:
engine, session = db.connect()
engine.echo = False

2023-03-27 10:07:17,397 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-27 10:07:17,397 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-27 10:07:17,398 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-27 10:07:17,398 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-27 10:07:17,399 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-27 10:07:17,399 INFO sqlalchemy.engine.Engine [raw sql] {}


## CSV with correct identification

In [41]:
df = pd.read_csv('../csv/list_genus_species_correct.csv', sep=';', index_col=None, header=0).astype(str)
df = df.replace('nan', None)
df

,kingdom,phylum,order,classe,family,genus,specific_epithet,infraspecific_epithet,scientific_name_authorship,kingdom_trusted,phylum_trusted,order_trusted,classe_trusted,family_trusted,genus_trusted,specific_epithet_trusted,infraspecific_epithet_trusted,scientific_name_authorship_trusted
0,None,None,None,None,None,Ottonia,anisum,None,Spreng.,None,None,None,None,None,Piper,anisum,None,(Spreng.) Angely
1,None,None,None,None,None,Ottonia,anisum,f. glabrior,Miq.,None,None,None,None,None,Piper,anisum,None,(Spreng.) Angely
2,None,None,None,None,None,Ottonia,armondii,None,Trel.,None,None,None,None,None,Piper,anisum,None,(Spreng.) Angely
3,None,None,None,None,None,Ottonia,burchellii,None,Trel.,None,None,None,None,None,Piper,anisum,None,(Spreng.) Angely
4,None,None,None,None,None,Ottonia,carpinifolia,f. hirtipedicellata,Yunck.,None,None,None,None,None,Piper,anisum,None,(Spreng.) Angely
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,None,None,None,None,None,Ottonia,brevistipitata,None,(C. DC.) Trel.,None,None,None,None,None,Piper,ovatum,var. hirtellum,C. DC.
114,None,None,None,None,None,Ottonia,leptostachya,var. brevistipitata,(C. DC.) E.F. Guim.,None,None,None,None,None,Piper,ovatum,var. hirtellum,C. DC.
115,None,None,None,None,None,Piper,frutescens,None,C. DC.,None,None,None,None,None,Piper,ovatum,var. hirtellum,C. DC.
116,None,None,None,None,None,Ottonia frutescens (C. DC.) Trel.,None,None,None,None,None,None,None,None,Piper,ovatum,var. hirtellum,C. DC.


## Genus level

In [15]:
old_genus = [['Sarcorhachis'], ['Pothomorphe'], ['Piperomia', 'Peperonia']]
new_genus = ['Manekia', 'Piper', 'Peperomia']

for g in zip(old_genus, new_genus):
    list_old_genus = g[0]
    new = g[1]
    for old in list_old_genus:
        session.query(DataTrustedIdentifier)\
            .filter(DataTrustedIdentifier.genus.__eq__(old))\
            .update(values={DataTrustedIdentifier.genus_trusted: new}, synchronize_session=False)

## Below genus level

In [60]:
for d in df[['genus', 'specific_epithet', 'infraspecific_epithet', 'scientific_name_authorship', 'genus_trusted', 'specific_epithet_trusted', 'infraspecific_epithet_trusted','scientific_name_authorship_trusted']].iterrows():
    print('genus (old) %s - (new) %s' % (d[1].genus, d[1].genus_trusted))
    print('specific_epithet (old) %s - (new) %s' % (d[1].specific_epithet, d[1].specific_epithet_trusted))
    print('infraspecific_epithet (old) %s - (new) %s' % (d[1].infraspecific_epithet, d[1].infraspecific_epithet_trusted))
    print('scientific_name_authorship (old) %s - (new) %s' % (d[1].scientific_name_authorship, d[1].scientific_name_authorship_trusted))

    columns=[
        DataTrustedIdentifier.genus,
        DataTrustedIdentifier.specific_epithet,
        sa.func.replace(sa.func.replace(DataTrustedIdentifier.infraspecific_epithet, 'f. ', ''), 'var. ', '').label('infraspecific_epithet'),
        sa.func.regexp_replace(DataTrustedIdentifier.scientific_name_authorship, '!| |(|)|.|&', '').label('scientific_name_authorship'),
    ]
    sub = session.query(*columns).subquery('sub')

    infraspecific_epithet = d[1].infraspecific_epithet
    if infraspecific_epithet:
        infraspecific_epithet = infraspecific_epithet.replace('f. ', '').replace('var. ', '')
        print('antes %s depois %s' % (d[1].infraspecific_epithet, infraspecific_epithet))

    scientific_name_authorship = d[1].scientific_name_authorship
    if scientific_name_authorship:
        scientific_name_authorship = re.sub('\W+', '', scientific_name_authorship)
        print('antes %s depois %s' % (d[1].scientific_name_authorship, scientific_name_authorship))


    session.query(DataTrustedIdentifier)\
        .filter(sa.and_(DataTrustedIdentifier.genus.__eq__(d[1].genus),
                        DataTrustedIdentifier.specific_epithet.__eq__(d[1].specific_epithet),
                        sa.or_(sub.c.infraspecific_epithet.__eq__(infraspecific_epithet),
                               sub.c.scientific_name_authorship.__eq__(scientific_name_authorship))))\
        .update(values={DataTrustedIdentifier.genus_trusted: d[1].genus_trusted,
                        DataTrustedIdentifier.specific_epithet_trusted: d[1].specific_epithet_trusted,
                        DataTrustedIdentifier.infraspecific_epithet_trusted: d[1].infraspecific_epithet_trusted,
                        DataTrustedIdentifier.scientific_name_authorship_trusted: d[1].scientific_name_authorship_trusted}, synchronize_session=False)

    session.commit()

genus (old) Ottonia - (new) Piper
specific_epithet (old) anisum - (new) anisum
infraspecific_epithet (old) None - (new) None
scientific_name_authorship (old) Spreng. - (new) (Spreng.) Angely
antes Spreng. depois Spreng
genus (old) Ottonia - (new) Piper
specific_epithet (old) anisum - (new) anisum
infraspecific_epithet (old) f. glabrior - (new) None
scientific_name_authorship (old) Miq. - (new) (Spreng.) Angely
antes f. glabrior depois glabrior
antes Miq. depois Miq
genus (old) Ottonia - (new) Piper
specific_epithet (old) armondii - (new) anisum
infraspecific_epithet (old) None - (new) None
scientific_name_authorship (old)  Trel. - (new) (Spreng.) Angely
antes  Trel. depois Trel
genus (old) Ottonia - (new) Piper
specific_epithet (old) burchellii - (new) anisum
infraspecific_epithet (old) None - (new) None
scientific_name_authorship (old) Trel. - (new) (Spreng.) Angely
antes Trel. depois Trel
genus (old) Ottonia - (new) Piper
specific_epithet (old) carpinifolia - (new) anisum
infraspecif

In [70]:
query = session.query(DataTrustedIdentifier)\
    .filter(sa.or_(DataTrustedIdentifier.genus_trusted.is_not(None),
                   DataTrustedIdentifier.specific_epithet_trusted.is_not(None),
                   DataTrustedIdentifier.infraspecific_epithet_trusted.is_not(None),
                   DataTrustedIdentifier.scientific_name_authorship_trusted.is_not(None)))\
    .all()

print('count of records in table %s updated: %d' % (DataTrustedIdentifier.__tablename__, len(query)))

count of records in table data_trusted_identifier updated 46


In [69]:
pd.DataFrame({
    'genus': [q.genus for q in query],
    'specific_epithet': [q.specific_epithet for q in query],
    'infraspecific_epithet': [q.infraspecific_epithet for q in query],
    'scientific_name_authorship': [q.scientific_name_authorship for q in query],
    'genus_new': [q.genus_trusted for q in query],
    'specific_epithet_new': [q.specific_epithet_trusted for q in query],
    'infraspecific_epithet_new': [q.infraspecific_epithet_trusted for q in query],
    'scientific_name_authorship_new': [q.scientific_name_authorship_trusted for q in query],
})

,genus,specific_epithet,infraspecific_epithet,scientific_name_authorship,genus_new,specific_epithet_new,infraspecific_epithet_new,scientific_name_authorship_new
0,Ottonia,propinqua,None,Kunth,Piper,grazielae,None,M. Carv.-Silva & E.F. Guim.
1,Piper,frutescens,None,C.DC.,Piper,ovatum,var. hirtellum,C. DC.
2,Ottonia,elocalis,None,Trel.,Piper,miquelianum,None,C. DC.
3,Ottonia,propinqua,None,Kunth,Piper,grazielae,None,M. Carv.-Silva & E.F. Guim.
4,Piper,corcovadense,None,C.DC.,Piper,grazielae,None,M. Carv.-Silva & E.F. Guim.
5,Ottonia,santa-annae,None,Trel.,Piper,miquelianum,None,C. DC.
6,Piper,quadratiovarium,None,Yunck.,Piper,hoffmannseggianum,None,Schult.
7,Ottonia,sampaioi,None,Vell.,Piper,grazielae,None,M. Carv.-Silva & E.F. Guim.
8,Ottonia,martiana,None,Miq.,Piper,miquelianum,None,C. DC.
9,Ottonia,martiana,None,Miq.,Piper,miquelianum,None,C. DC.


## Close connection

In [18]:
session.close()
engine.dispose()